In [23]:
from gensim import corpora, models
from nltk.corpus import sinica_treebank
import jieba
import re

In [24]:
# Pre-process
with open('../source/cn_stopwords.txt', 'r', encoding='utf-8') as f:
    stopwords = f.read().splitlines()

s = "此外，公司拟对全资子公司吉林欧亚置业有限公司增资4.3亿元，增资后，吉林欧亚置业注册资本由7000万元增加到5亿元。吉林欧亚置业主要经营范围为房地产开发及百货零售等业务。目前在建吉林欧亚城市商业综合体项目。2013年，实现营业收入0万元，实现净利润-139.13万元。"

words = [word for word in jieba.lcut(re.sub(r'[\d\.-]+', '', s)) if len(word) > 1 and not word in stopwords ]

print(words)

['公司', '全资', '子公司', '吉林', '欧亚', '置业', '有限公司', '增资', '亿元', '增资', '吉林', '欧亚', '置业', '注册资本', '万元', '增加', '亿元', '吉林', '欧亚', '置业', '主要', '经营范围', '房地产', '开发', '百货', '零售', '业务', '目前', '在建', '吉林', '欧亚', '城市', '商业', '综合体', '项目', '实现', '营业', '收入', '万元', '实现', '净利润', '万元']


In [25]:
# Keyword Extraction using gensim LSA

dictionary = corpora.Dictionary([words])
corpus = [dictionary.doc2bow(words)]
lsa = models.LsiModel(corpus, id2word=dictionary, num_topics=1)

for topic_id, topic in lsa.print_topics(num_words=5):
    keywords = [word.rstrip('"').split(r'*"') for word in topic.split(" + ")]

weight = [float(word[0]) for word in keywords]
keywords = [word[1] for word in keywords]

for i in range(len(keywords)):
    print(f"{keywords[i]} {weight[i]}")

吉林 0.436
欧亚 0.436
置业 0.327
万元 0.327
增资 0.218


In [26]:
# Keyword Extraction using gensim LDA

dictionary = corpora.Dictionary([words])
corpus = [dictionary.doc2bow(words)]
lda = models.LdaModel(corpus, id2word=dictionary, num_topics=1)

for topic_id, topic in lda.print_topics(num_words=5):
    keywords = [word.rstrip('"').split(r'*"') for word in topic.split(" + ")]

weight = [float(word[0]) for word in keywords]
keywords = [word[1] for word in keywords]

for i in range(len(keywords)):
    print(f"{keywords[i]} {weight[i]}")

吉林 0.07
欧亚 0.07
万元 0.056
置业 0.056
实现 0.042


In [27]:
# Keyword Extraction using jieba TextRank

for x, w in jieba.analyse.textrank(s, withWeight=True)[:5]:
    print('%s %s' % (x, w))

吉林 1.0
欧亚 0.9966893354178172
置业 0.6434360313092776
实现 0.5898606692859626
收入 0.43677859947991454


In [34]:
# Keyword Extraction using TF-IDF by hand

idf = {}
with open("../source/idf.txt.big", 'r', encoding='utf-8') as f:
    lines = f.read().splitlines()
    for line in lines:
        word, value = line.split()
        idf[word] = float(value)

def TF_IDF(words, idf):
    tfidf = {}
    tf = {}
    for word in words:
        if word in tf:
            tf[word] += 1
        else:
            tf[word] = 1
    for word in tf:
        tf[word] = tf[word] / len(words)
        tfidf[word] = tf[word] * idf.get(word, 14)
    return tfidf

tfidf = TF_IDF(words, idf)
for word in sorted(tfidf.items(), key=lambda x: x[1], reverse=True)[:5]:
    print(word[0], word[1])

吉林 1.3333333333333333
欧亚 0.8169207307466667
增资 0.6666666666666666
实现 0.6666666666666666
置业 0.5468936250935714
